# [1] 시나리오
상황: 무탠다드 그로스팀이 요청했습니다.

- "신규 사용자의 첫 구매 전환율이 기대보다 낮습니다. 앱 실행부터 결제 완료까지, 어떤 단계에서 가장 많이 이탈하는지 분석해 주세요."

## (1) 데이터 확인하기

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import plotly.graph_objects as go

In [4]:
path = '/content/drive/MyDrive/새싹2기/python_DA/data/'

In [7]:
df = pd.read_csv(path + 'mutandard_funnel_data.csv')

In [8]:
df.shape

(46874, 4)

In [9]:
df.dtypes

,0
user_id,object
event_name,object
channel,object
timestamp,object


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46874 entries, 0 to 46873
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     46874 non-null  object
 1   event_name  46874 non-null  object
 2   channel     46874 non-null  object
 3   timestamp   46874 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [12]:
df.isnull().sum()

,0
user_id,0
event_name,0
channel,0
timestamp,0


In [11]:
df.head()

,user_id,event_name,channel,timestamp
0,u_00001,app_open,organic,2024-01-01 00:00:00
1,u_00001,search,organic,2024-01-01 00:13:00
2,u_00001,view_product,organic,2024-01-01 00:08:00
3,u_00001,view_review,organic,2024-01-01 00:33:00
4,u_00002,app_open,youtube_review,2024-01-01 00:03:00


## 과제1 : 퍼널 단계 정의 및 사용자수 집계

In [14]:
df['event_name'].value_counts()

,count
event_name,
app_open,15000
search,11217
view_product,8495
view_review,5126
add_to_cart,3276
begin_checkout,2189
payment_complete,1571


In [16]:
funnel = ['app_open', 'search', 'view_product', 'view_review', 'add_to_cart', 'begin_checkout', 'payment_complete']

agg =df.groupby('event_name')['user_id'].nunique().reindex(funnel)
funnel_df = agg.reset_index()
funnel_df.columns = ['stage', 'user_count']

In [18]:
funnel_df

,stage,user_count
0,app_open,15000
1,search,11217
2,view_product,8495
3,view_review,5126
4,add_to_cart,3276
5,begin_checkout,2189
6,payment_complete,1571


## 과제 2: 전환율 · 이탈률 계산

In [19]:
# 단계별 전환율 (직접 작성)
funnel_df['conversion_rate'] = (funnel_df['user_count'] / funnel_df['user_count'].shift(1) * 100).round(1)

In [20]:
# 전체 전환율 (app_open 대비)
funnel_df['overall_rate'] = (funnel_df['user_count'] / funnel_df['user_count'].iloc[0] * 100).round(1)

In [21]:
# 이탈률
funnel_df['dropoff_rate'] = (100 - funnel_df['conversion_rate']).round(1)

In [22]:
funnel_df

,stage,user_count,conversion_rate,overall_rate,dropoff_rate
0,app_open,15000,NaN,100.0,NaN
1,search,11217,74.8,74.8,25.2
2,view_product,8495,75.7,56.6,24.3
3,view_review,5126,60.3,34.2,39.7
4,add_to_cart,3276,63.9,21.8,36.1
5,begin_checkout,2189,66.8,14.6,33.2
6,payment_complete,1571,71.8,10.5,28.2


## 과제3: 퍼널 시각화

In [35]:
fig = go.Figure(go.Funnel(
    y = funnel_df['stage'],
    x = funnel_df['user_count'],
    textposition= 'inside',
    textinfo = 'value+percent initial+percent previous',
    opacity = 0.85,
marker = {
        "color": ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", '#01B14D'],
        "line": {"width": 1, "color": "#333333"}
    },
connector = {
        "line": {"color": "gray", "dash": "dot", "width": 2}
    }
))

fig.update_layout(
    title={
        'text': "무탠다드 신규 사용자 첫 구매 펀러",
        'y': 0.9, 'x': 0.5,
        'xanchor': 'center', 'yanchor': 'top'
    },
    font={'family': "Arial, sans-serif", 'size': 12, 'color': "black"}
)

fig.show()

In [36]:
bottleneck_idx = funnel_df['dropoff_rate'].idxmax()
bottleneck = funnel_df.loc[bottleneck_idx]

print(f"병목 단계 : {bottleneck['stage']}")
print(f"이탈률 : {bottleneck['dropoff_rate']}%")

병목 단계 : view_review
이탈률 : 39.7%


## 과제 4: 병목 지점 해석

> 가설 수립: 패션 커머스의 특성을 고려할 때, 이 단계에서 이탈하는 이유는?
- 가설 1: 내 체형에 맞는 리뷰가 없음
- 가설 2: 포토 리뷰가 없어 실착 핏을 확인 불가
- 가설 3: 리뷰에 대한 신뢰신호가 부족(인증 구매자 표시/ 별점 이외에 세분화 지표 필요)
\

## 과제 5: 세그먼트별 퍼널 비교

- 과제 A: 유입 채널별(오가닉/인스타그램 광고/유튜브 리뷰) 퍼널을 비교하는 코드를 작성하세요.


In [40]:
df.head()

,user_id,event_name,channel,timestamp
0,u_00001,app_open,organic,2024-01-01 00:00:00
1,u_00001,search,organic,2024-01-01 00:13:00
2,u_00001,view_product,organic,2024-01-01 00:08:00
3,u_00001,view_review,organic,2024-01-01 00:33:00
4,u_00002,app_open,youtube_review,2024-01-01 00:03:00


In [39]:
df['channel'].value_counts()

,count
channel,
organic,21763
instagram_ad,14177
youtube_review,10934


In [43]:
channel_funnel = df.groupby(['channel', 'event_name'])['user_id'].nunique().unstack(fill_value=0)
channel_funnel = channel_funnel[funnel]
print(channel_funnel)


# 채널별 전환율 계산
channels = ['organic', 'instagram_ad', 'youtube_review']
for ch in channels:
    row = channel_funnel.loc[ch]
    vp_vr = row['view_review'] / row['view_product'] * 100
    atc_pay = row['payment_complete'] / row['add_to_cart'] * 100
    print(f"{ch:20}: view_product→view_review {vp_vr:.1f}% | add_to_cart→payment {atc_pay:.1f}%")

event_name      app_open  search  view_product  view_review  add_to_cart  \
channel                                                                    
instagram_ad        5163    3748          2707         1184          681   
organic             6844    5137          3941         2483         1567   
youtube_review      2993    2332          1847         1459         1028   

event_name      begin_checkout  payment_complete  
channel                                           
instagram_ad               416               278  
organic                   1033               758  
youtube_review             740               535  
organic             : view_product→view_review 63.0% | add_to_cart→payment 48.4%
instagram_ad        : view_product→view_review 43.7% | add_to_cart→payment 40.8%
youtube_review      : view_product→view_review 79.0% | add_to_cart→payment 52.0%


- 유튜브 리뷰 유입이 전체적으로 높은 이유는?
  
  : 영상을 통해 자세한 제품과 유튜버의 솔직한 리뷰를 듣고 들어오기 때문에 제품에 대한 불안감이 적고 전환율이 높음
- 인스타그램 광고 유입의 문제점과 개선안은?

  : 인스타그램은 겉보기 사진에 따른 얕은 유입이 많아 전환율이 낮음

# [2] 액션 플랜 작성
- 병목 분석을 바탕으로 PM에게 보고할 액션 플랜을 작성하세요.
> 분석 보고서 구성:
1. 현황 요약
- 전체 전환율: app_open → payment_complete = 10%
- 주요 병목: view_product => view_review - 이탈률 40%
2. 문제 원인 (가설)
- 가설 1: 내 체형에 맞는 리뷰, 자세한 포토 리뷰 부족
- 가설 2: 별점 외의 제품에 대한 신뢰를 판단할 만한 지표 부족
3. 개선안 (Action Items)
- [P0] 우선순위 1: 가입 시에 내 체형 기입할 수 있는 `회원가입 기능 추가`
- [P1] 우선순위 2: 리뷰에 대한 간략한 요약을 카드 형식으로 `긍정/부정 비율에 대해 표시해줌`
- [P2] 우선순위 3: 인스타그램 광고를 위한 콘텐츠가 아닌, 콘텐츠 자체가 광고가 되게 만드는 `Recycling 전략 수립`
4. 성공 측정
- 주요 KPI:
  - 전체 CVR(전환율)
  - view_product =>  view_review 전환율
  - view_review => add_to_cart 전환율
- 목표 수치:
  - 병목 구간 전환율 5% 이상 개선
- 측정 기간:
  - 월 단위(4주) A/B 테스트